In [94]:
import numpy as np
import pandas as pd
import config
from datetime import datetime

import utm

In [95]:
# Import data
crime_data = pd.read_csv('../data/crimedata_allneighbourhoods_allyears.csv')

In [96]:
# Rename columns X and Y to UTM_X and UTM_Y
crime_data.rename(columns={'X':'UTM_E','Y':'UTM_N'},inplace=True)

Some crime instances have location coordinates NaN, suggesting missing location data. These instances have been removed.

In [97]:
# Remove for NaNs among relevant columns
processed_crime_data = crime_data[~crime_data.isna().any(axis=1)]

Some crime types (*Homicide* and *Offense Against a Person*) have instances with location coordinates mentioned as 0 due to privacy concerns. To address this, we have generated random location coordinates within the provided neighbourhood in order to retain these instances.

In [98]:
# For now removing these crime types
processed_crime_data = processed_crime_data[~processed_crime_data['TYPE'].isin(['Homicide','Offence Against a Person'])]


In [99]:
# Convert UTM coordinates to Latitude and Longitude
def utm2latlong(utm_x, utm_y, utm_zone_no, utm_zone_ltr):
    lat, long = utm.to_latlon(utm_x, utm_y, utm_zone_no, utm_zone_ltr)
    return lat,long

In [100]:
# Convert UTM coordinates to latitude and longitude
processed_crime_data['LAT'], processed_crime_data['LONG'] = utm2latlong(processed_crime_data['UTM_E'],
                                                                        processed_crime_data['UTM_N'],
                                                                        config.UTM_ZONE_NO, 
                                                                        config.UTM_ZONE_LTR)

In [101]:
# Function to create bins
def getBins(min_,max_,n_bins):
    bins = np.linspace(start=min_, stop=max_, num=n_bins+1)
    return bins

In [102]:
## Generate latitude and longitude bins

# Number of bins = bounding box length/ length of each cell
n_bins = int(config.BB_DIST/config.BB_CELL_LEN)
    
# Get minimum and maxiumum values for longitudes and latitudes of bounding box vertices
min_lat = min(config.BB_VERTICES.values(), key = lambda x: x['lat'])['lat']
max_lat = max(config.BB_VERTICES.values(), key = lambda x: x['lat'])['lat']

min_long = min(config.BB_VERTICES.values(), key = lambda x: x['long'])['long']
max_long = max(config.BB_VERTICES.values(), key = lambda x: x['long'])['long']

# Divide bounding box into bins
lat_bins = getBins(min_=min_lat, max_=max_lat, n_bins=n_bins)
long_bins = getBins(min_=min_long, max_=max_long, n_bins=n_bins)

In [103]:
# Function to generate cell coordinates
def getCellLocs(lats,longs,lat_bins,long_bins, correction=False):
    
    # Assign x coordinate of cell. X coordinates can range from 1 to lat_bins and are upper bound.
    # Location coordinates laying outside the bounding box are labelled as either 0 or len(lat_bins)
    cell_x = np.digitize(lats,lat_bins,right=True)
    
    # Assign x coordinate of cell. Y coordinates can range from 1 to long_bins and are upper bound.
    # Location coordinates laying outside the bounding box are labelled as either 0 or len(long_bins)
    cell_y = np.digitize(longs,long_bins,right=True)
    
    if correction == True:
        cell_x_corr = [i-1 if i==len(lat_bins) else i+1 if i==0 else i for i in cell_x] 
        cell_y_corr = [i-1 if i==len(long_bins) else i+1 if i==0 else i for i in cell_y]
        
        return cell_x_corr, cell_y_corr
    
    elif correction == False:
        cell_x_excl = [-1 if i==0 or i==len(lat_bins) else i for i in cell_x]
        cell_y_excl = [-1 if i==0 or i==len(long_bins) else i for i in cell_y]
        
        return cell_x_excl, cell_y_excl

In [104]:
## Generate cell coordinates
processed_crime_data['CELL_X'],processed_crime_data['CELL_Y'] = getCellLocs(lats=processed_crime_data['LAT'].values,
                                                                            longs=processed_crime_data['LONG'].values,
                                                                            lat_bins=lat_bins,
                                                                            long_bins=long_bins,
                                                                            correction=True)
# Filter out rows with cell coordinate values as -1 (occur in the case on correction == False)
processed_crime_data = processed_crime_data[(processed_crime_data['CELL_X'] > -1) & 
                                            (processed_crime_data['CELL_Y'] > -1)]

In [105]:
# Function to fetch crime date from day, month and year
def getDate(day, month, year):
    dt = datetime(year, month, day)
    date = dt.date()
    return date

In [106]:
# Fetch crime date
processed_crime_data['DATE'] = processed_crime_data.apply(lambda row: getDate(day=row['DAY'],
                                                                              month=row['MONTH'],
                                                                              year=row['YEAR']),axis=1)

# Order by crime date
processed_crime_data.sort_values(by='DATE', inplace=True)

In [107]:
# Fetch crime broad category
processed_crime_data['CAT'] = processed_crime_data.apply(lambda row: config.TYPE2CAT[row['TYPE']],axis=1)

In [108]:
# Save processed crime data as pickle
processed_crime_data.to_pickle('../data/processed_crime_data.pkl')

Still working on the code below

In [88]:
crime_pivot = processed_crime_data.pivot_table(values='TYPE', index=['DATE','CAT'], columns=['CELL_X','CELL_Y'], aggfunc='count')

In [89]:
crime_pivot.columns = crime_pivot.columns.to_flat_index()

In [74]:
crime_pivot.reset_index(inplace=True)

In [93]:
crime_pivot.index

MultiIndex([(2003-01-01,   'Break and Enter'),
            (2003-01-01,          'Mischief'),
            (2003-01-01,             'Theft'),
            (2003-01-01, 'Vehicle Collision'),
            (2003-01-02,   'Break and Enter'),
            (2003-01-02,          'Mischief'),
            (2003-01-02,             'Theft'),
            (2003-01-02, 'Vehicle Collision'),
            (2003-01-03,   'Break and Enter'),
            (2003-01-03,          'Mischief'),
            ...
            (2022-02-09,          'Mischief'),
            (2022-02-09,             'Theft'),
            (2022-02-09, 'Vehicle Collision'),
            (2022-02-10,   'Break and Enter'),
            (2022-02-10,          'Mischief'),
            (2022-02-10,             'Theft'),
            (2022-02-10, 'Vehicle Collision'),
            (2022-02-11,   'Break and Enter'),
            (2022-02-11,          'Mischief'),
            (2022-02-11,             'Theft')],
           names=['DATE', 'CAT'], length=27

In [87]:
processed_crime_data

,TYPE,YEAR,MONTH,DAY,HOUR,MINUTE,HUNDRED_BLOCK,NEIGHBOURHOOD,UTM_E,UTM_N,LAT,LONG,CELL_X,CELL_Y,DATE,CAT
144629,Mischief,2003,1,1,1,45,21XX W 2ND AVE,Kitsilano,488770.5190,5.457479e+06,49.269968,-123.154365,20,7,2003-01-01,Mischief
709741,Theft from Vehicle,2003,1,1,0,1,X NK_LOC ST,Strathcona,492757.4868,5.458792e+06,49.281843,-123.099582,22,15,2003-01-01,Theft
666010,Theft from Vehicle,2003,1,1,3,0,8XX HOMER ST,Central Business District,491473.3602,5.458499e+06,49.279183,-123.117232,22,13,2003-01-01,Theft
61501,Break and Enter Residential/Other,2003,1,1,0,20,19XX PARKER ST,Grandview-Woodland,495279.4615,5.458106e+06,49.275692,-123.064898,21,20,2003-01-01,Break and Enter
709740,Theft from Vehicle,2003,1,1,0,0,X NK_LOC ST,Strathcona,492757.4868,5.458792e+06,49.281843,-123.099582,22,15,2003-01-01,Theft
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16774,Break and Enter Commercial,2022,2,11,0,19,20XX W BROADWAY AVE,Kitsilano,488897.4913,5.456790e+06,49.263775,-123.152600,18,8,2022-02-11,Break and Enter
307896,Other Theft,2022,2,11,4,0,13XX W 12TH AVE,Fairview,490190.8459,5.456445e+06,49.260692,-123.134815,17,10,2022-02-11,Theft
174690,Mischief,2022,2,11,3,17,4XX W CORDOVA ST,Central Business District,491996.9388,5.459041e+06,49.284070,-123.110045,23,14,2022-02-11,Mischief
50042,Break and Enter Residential/Other,2022,2,11,0,8,12XX SEYMOUR ST,Central Business District,490806.8322,5.458107e+06,49.275649,-123.126387,21,11,2022-02-11,Break and Enter


In [46]:
inds

array([0, 4, 3, 2])